# Laboratorio 1 - Análisis Exploratorio de Datos
### Indicadores Macroeconómicos Mundiales

Grupo 7:
- Ariel Espinoza (202373060-2)
- Martin García (202373068-8)
- Williams Ajata (202204067-k)
- Catalina Yañez (202273010-2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('default')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Carga y Contextualización del Dataset

In [ ]:
df = pd.read_csv("world_bank_data_2025.csv")
df.head()

**Contexto del dataset**  
- Contiene indicadores macroeconómicos de diversos países a lo largo de varios años.  
- Fuente: Banco Mundial y otras fuentes internacionales.  
- Motivación:  Nos intereso este dataset ya que es parte de la vida real y permite realizar un análisis de como los países se enfrentan a crisis o crecimiento económico, de esta forma podemos relacionar las variables y concluir las posibles causantes de estas.


## 2. Limpieza y Preparación de Datos

**Información inicial del dataset**

In [ ]:
print(df.info())

**Valores faltantes**

In [ ]:
null_info = pd.DataFrame({
    "Valores_Nulos": df.isnull().sum(),
    "Porcentaje_Nulos": (df.isnull().sum() / len(df)) * 100
})
null_info[null_info["Valores_Nulos"] > 0]

**Tratar los valores faltantes**

In [ ]:
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

**Normalizar texto**

In [ ]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

**Eliminar Duplicados**

In [ ]:
df = df.drop_duplicates()

**Comprobar información general**

In [ ]:
print(df.info())

**Comprobar Valores Nulos**

In [ ]:
null_info = pd.DataFrame({
    "Valores_Nulos": df.isnull().sum(),
    "Porcentaje_Nulos": (df.isnull().sum() / len(df)) * 100
})
null_info[null_info["Valores_Nulos"] > 0]

**Comprobar de valores atípicos**

In [ ]:
df[(df["Unemployment Rate (%)"] < 0) | (df["Unemployment Rate (%)"] > 50)]

## 3. Análisis Exploratorio de Datos


**Identificar columnas numéricas y categóricas**

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns

print("Variables numéricas:", list(num_cols))
print("Variables categóricas:", list(cat_cols))

**Estadísticas de las numéricas**

In [ ]:
df[num_cols].describe().T

**Resumen de variables categóricas**

In [ ]:
for c in cat_cols:
    print(f"\n{c}:")
    print(df[c].value_counts().head(5))

**Identificación de outliers**

In [ ]:
def detectar_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5*IQR
    limite_superior = Q3 + 1.5*IQR
    outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)][col]
    return outliers

for col in num_cols:
    outliers = detectar_outliers(col)
    print(f"{col}: {len(outliers)} outliers detectados")

**Boxplots**

In [ ]:
df[num_cols].boxplot(figsize=(12,6))
plt.title("Boxplots - Variables numéricas")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
df[num_cols].boxplot()
plt.yscale("log")
plt.title("Boxplots con escala logarítmica")
plt.xticks(rotation=45)
plt.show()

**Top 10 países por PIB promedio**

In [ ]:
df.groupby("country_name")["GDP (Current USD)"].mean().sort_values(ascending=False).head(10)

**Top 10 países por crecimiento promedio**

In [ ]:
df.groupby("country_name")["GDP Growth (% Annual)"].mean().sort_values(ascending=False).head(10)

## 4. Visualizaciones


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b']

# Evolución PIB mundial
yearly_gdp = df.groupby("year")["GDP (Current USD)"].mean()
axes[0,0].plot(yearly_gdp.index, yearly_gdp.values, marker="o", color=colors[0])
axes[0,0].set_title("Evolución del PIB Mundial Promedio", fontweight="bold")
axes[0,0].set_xlabel("Año")
axes[0,0].set_ylabel("PIB Promedio (USD)")
axes[0,0].grid(True, alpha=0.3)

# Top 10 PIB
top_10 = df.groupby("country_name")["GDP (Current USD)"].mean().nlargest(10)
axes[0,1].barh(top_10.index, top_10.values, color=colors[:10])
axes[0,1].set_title("Top 10 Países por PIB Promedio", fontweight="bold")
axes[0,1].set_xlabel("PIB Promedio (USD)")
axes[0,1].set_ylabel("Países")

# Relación PIB per cápita vs desempleo
sample_df = df.sample(min(1000,len(df)))
axes[1,0].scatter(sample_df["GDP per Capita (Current USD)"], sample_df["Unemployment Rate (%)"], alpha=0.6, color=colors[2])
axes[1,0].set_title("Relación PIB per Cápita vs Desempleo", fontweight="bold")
axes[1,0].set_xlabel("PIB per Cápita (USD)")
axes[1,0].set_ylabel("Tasa de Desempleo (%)")
axes[1,0].set_xscale("log")

# Distribución crecimiento económico
growth_data = df["GDP Growth (% Annual)"].dropna()
axes[1,1].hist(growth_data, bins=30, color=colors[3], edgecolor="black", alpha=0.7)
axes[1,1].set_title("Distribución del Crecimiento Económico", fontweight="bold")
axes[1,1].set_xlabel("Crecimiento del PIB (% Anual)")
axes[1,1].set_ylabel("Frecuencia")
axes[1,1].grid(True, alpha=0.3)

plt.suptitle("Análisis Exploratorio - Indicadores Macroeconómicos Mundiales", fontsize=16, fontweight="bold")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**Matriz de correlación**

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr = df[numeric_cols].corr()

plt.figure(figsize=(10,8))
im = plt.imshow(corr, cmap="coolwarm", vmin=-1, vmax=1)
plt.xticks(range(len(corr)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr)), corr.columns)
plt.colorbar(im, label="Coef. Correlación")
plt.title("Matriz de Correlación - Variables Económicas")
plt.show()

## Gráficos evolución Mundial

In [ ]:
world_growth = df.groupby("year")["GDP Growth (% Annual)"].mean()
world_unemployment = df.groupby("year")["Unemployment Rate (%)"].mean()
world_inflation = df.groupby("year")["Inflation (CPI %)"].mean()
fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Crecimiento económico
axes[0].plot(world_growth.index, world_growth.values, marker="o", color="green", label="Crecimiento económico mundial")
axes[0].set_title("Evolución del crecimiento económico mundial (% promedio anual)", fontweight="bold")
axes[0].set_ylabel("GDP Growth (%)")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Desempleo
axes[1].plot(world_unemployment.index, world_unemployment.values, marker="o", color="red", label="Desempleo mundial")
axes[1].set_title("Evolución del desempleo mundial (% promedio anual)", fontweight="bold")
axes[1].set_ylabel("Unemployment Rate (%)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Inflación
axes[2].plot(world_inflation.index, world_inflation.values, marker="o", color="blue", label="Inflación mundial")
axes[2].set_title("Evolución de la Inflación Mundial (CPI %)", fontweight="bold")
axes[2].set_xlabel("Año")
axes[2].set_ylabel("Inflación (CPI %)")
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Selección de Variables
- **Principales**: `Unemployment Rate (%)`, `GDP Growth (% Annual)`  
- **Secundarias**: `Inflation (CPI %)`, `GDP (Current USD)`  

**Justificación**:  
- PIB y crecimiento son indicadores centrales del desempeño económico.  
- Inflación y desempleo son críticos en política económica.  
- Existen correlaciones interesantes entre ellas. 

Nota: Idea confirmada por ChatGPT: en base a nuestro trabajo, considerarias que el gdp(current usd), y gdp growth% anual como principales, y la inflation CPI% y Unemployment rate% como secundarias es una factible seleccion de variables para poder hondar en nuestra pregunta de investigacion?

## 6. Pregunta de Investigación
Analizando los datos, nos dimos cuenta de que los paises más desarrolados tienen in PIB mucho más elevado que el de aquellos que no, en especial aquellas que pueden ser consideradas potencias mundiales, cómo China y EE.UU. Además, el crecimiento del PIB en el mundo empezó a crecer en gran manera en 2020, lo que justamente coincide con la pandemia, lo que nos deja con la idea de que algunos paises pudieron aprovechar muy bien la situación para poder crecer y desarrollarse durante este periodo, teninendo como consecuencia, que su PIB aumente. Sin embargo, siguen habiendo paises que se quedaron algo atras en comparación a otros, dando como resultado que tengan una tasa de desempleo más alta, ya que no supieron adaptarse tan bien como otros, evidenciando que, mientras más alta la tasa de crecimiento del PIB, la empleabilidad aumenta. Aunque, hay que destacar que, aunque algunos paises tengan un PIB muy alto, este no tiene una tasa de crecimiento tan alta como otros paises que tienen menos PIB.

Con esto en mente, no planteamos la siguiente pregunta de investigación:

**¿La tasa de crecimiento del PIB influye mucho más que la inflación y el PIB actual de un país, en su tasa de desempleo?**

Hipotesis de la pregunta planteada:

**La tasa de crecimiento sí influye más que el resto de variables en cuanto a la tasa de desempleo, siendo el PIB actual una forma de verificarlo y la inflación un factor que restrasaría este crecimiento.**

# Laboratorio 2 - Simulación e Inferencia

## 1. Estimación puntual

**Necesario ejecutar el código del laboratorio anteriror para evitar problemas**

In [ ]:
pip install scipy scikit-learn

In [ ]:
import scipy.stats as stats
from scipy.stats import t, norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

def calcular_estimadores_puntuales(data, variable_name):
    data_clean = data.dropna()
    n = len(data_clean)
    
    if n == 0:
        return None

    estimadores = {
        'n': n,
        'media_muestral': np.mean(data_clean),
        'mediana': np.median(data_clean),
        'varianza_muestral': np.var(data_clean, ddof=1),
        'desviacion_estandar': np.std(data_clean, ddof=1),
        'error_estandar_media': np.std(data_clean, ddof=1) / np.sqrt(n),
        'coef_variacion': (np.std(data_clean, ddof=1) / np.mean(data_clean)) * 100 if np.mean(data_clean) != 0 else np.nan,
        'minimo': np.min(data_clean),
        'maximo': np.max(data_clean),
        'q1': np.percentile(data_clean, 25),
        'q3': np.percentile(data_clean, 75),
        'asimetria': stats.skew(data_clean),
        'curtosis': stats.kurtosis(data_clean)
    }
    
    alpha = 0.05
    t_critical = t.ppf(1 - alpha/2, df=n-1)
    margin_error = t_critical * estimadores['error_estandar_media']
    
    estimadores['ic_95_inferior'] = estimadores['media_muestral'] - margin_error
    estimadores['ic_95_superior'] = estimadores['media_muestral'] + margin_error
    
    return estimadores

variables_principales = [
    'GDP Growth (% Annual)',
    'Inflation (CPI %)', 
    'GDP (Current USD)',
    'Unemployment Rate (%)'
]

print("1. Variables Seleccionadas")
print("Basándome en nuestra pregunta de investigación:")
print("¿La tasa de crecimiento del PIB influye mucho más que la inflación y el PIB actual de un país, en su tasa de desempleo?")
for i, var in enumerate(variables_principales, 1):
    tipo = 'dependiente' if 'Unemployment' in var else 'independiente'
    print(f"{i}. {var}: Variable {tipo}")

resultados_estimacion = {}

print()

print("2. Estimadores Puntuales por Variable")

for variable in variables_principales:
    print(f" {variable}")
    
    estimadores = calcular_estimadores_puntuales(df[variable], variable)
    resultados_estimacion[variable] = estimadores
    
    if estimadores is not None:
        print(f"- Tamaño de muestra (n): {estimadores['n']:,}")
        print(f"- Media muestral (μ̂): {estimadores['media_muestral']:.4f}")
        print(f"- Desviación estándar (s): {estimadores['desviacion_estandar']:.4f}")
        print(f"- Error estándar de la media: {estimadores['error_estandar_media']:.4f}")
        print(f"- Coeficiente de variación: {estimadores['coef_variacion']:.2f}%")
        print(f"- Intervalo de confianza 95%: [{estimadores['ic_95_inferior']:.4f}, {estimadores['ic_95_superior']:.4f}]")
        
        # Interpretación económica
        if 'GDP Growth' in variable:
            print("- Interpretación:** Crecimiento económico promedio de los países en la muestra")
        elif 'Inflation' in variable:
            print("- Interpretación:** Tasa de inflación promedio, valores 2-3% son considerados saludables")
        elif 'GDP (Current USD)' in variable:
            print("- Interpretación: Tamaño económico promedio de los países")
        elif 'Unemployment' in variable:
            print("- Interpretación: Tasa de desempleo promedio, valores < 5% indican mercados laborales saludables")
        print()

print("3. Tabla Comparativa de Estimadores")

comparison_data = []
for variable in variables_principales:
    if resultados_estimacion[variable] is not None:
        est = resultados_estimacion[variable]
        comparison_data.append({
            'Variable': variable.replace(' (%)', '').replace(' (Current USD)', ''),
            'Media': f"{est['media_muestral']:.4f}",
            'Desv.Std': f"{est['desviacion_estandar']:.4f}",
            'Error Std': f"{est['error_estandar_media']:.4f}",
            'CV (%)': f"{est['coef_variacion']:.2f}",
            'IC 95%': f"[{est['ic_95_inferior']:.3f}, {est['ic_95_superior']:.3f}]"
        })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print()

print("4. Validación de Supuestos - Test de Normalidad")

def test_normalidad_simple(data, alpha=0.05):
    data_clean = data.dropna()
    n = len(data_clean)
    
    if n < 3:
        return None
    
    if n > 5000:
        statistic, p_value = stats.kstest(data_clean, 'norm', 
                                        args=(np.mean(data_clean), np.std(data_clean)))
        test_name = "Kolmogorov-Smirnov"
    else:
        statistic, p_value = stats.shapiro(data_clean)
        test_name = "Shapiro-Wilk"
    
    return {
        'test': test_name,
        'p_value': p_value,
        'es_normal': p_value > alpha
    }

for variable in variables_principales:
    test_result = test_normalidad_simple(df[variable])
    if test_result is not None:
        print(f"{variable}:")
        print(f"- Test: {test_result['test']}")
        print(f"- p-valor: {test_result['p_value']:.4f}")
        print(f"- ¿Es normal?: {'Sí' if test_result['es_normal'] else 'No'} (α = 0.05)")
        print()

print("5. Estimación de Parámetros del Modelo Econométrico")
print("Modelo propuesto: Unemployment = β₀ + β₁(GDP_Growth) + β₂(Inflation) + β₃(log(GDP)) + ε")

# Preparar datos para el modelo
model_data = df[variables_principales].copy()
model_data = model_data.dropna()

print(f"Observaciones disponibles: {len(model_data):,}")

# Transformar GDP a logaritmo
model_data['log_GDP'] = np.log(model_data['GDP (Current USD)'])

# Variables del modelo
X = model_data[['GDP Growth (% Annual)', 'Inflation (CPI %)', 'log_GDP']]
y = model_data['Unemployment Rate (%)']

# Estimación MCO
modelo = LinearRegression()
modelo.fit(X, y)

# Predicciones y estadísticos
y_pred = modelo.predict(X)
r_squared = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

# Estimadores del modelo
beta_0 = modelo.intercept_
betas = modelo.coef_

print()

print("Estimadores puntuales del modelo:")
print(f"- β₀ (Intercepto): {beta_0:.4f}")
print(f"- β₁ (GDP Growth): {betas[0]:.4f}")
print(f"- β₂ (Inflation): {betas[1]:.4f}")
print(f"- β₃ (log GDP): {betas[2]:.4f}")
print(f"- R²: {r_squared:.4f}")
print(f"- RMSE: {rmse:.4f}")

print()

print("Interpretación económica:")
print(f"- Por cada 1% de aumento en el crecimiento del PIB, el desempleo {'disminuye' if betas[0] < 0 else 'aumenta'} en {abs(betas[0]):.4f} puntos porcentuales")
print(f"- Por cada 1% de aumento en la inflación, el desempleo {'disminuye' if betas[1] < 0 else 'aumenta'} en {abs(betas[1]):.4f} puntos porcentuales")
print(f"- El modelo explica {r_squared*100:.2f}% de la variabilidad en la tasa de desempleo")

print()

print("6. Propiedades de los Estimadores")
print("Media Muestral (X̄):")
print("- Insesgadez: E[X̄] = μ")
print("- Consistencia: Converge al parámetro poblacional")
print("- Eficiencia: Varianza mínima (BLUE)")
print("Varianza Muestral (s²):")
print("- Insesgadez: E[s²] = σ² (con corrección n-1)")
print("- Consistencia: Converge a la varianza poblacional")
print("Estimadores MCO:")
print("- Insesgadez: E[β̂] = β bajo supuestos de Gauss-Markov")
print("- Eficiencia: BLUE bajo supuestos clásicos")
print("- Consistencia: Convergen a parámetros verdaderos")

print()

print("7. Conclusiones de la Estimación Puntual")
print("a). Estimadores implementados: Se calcularon estimadores puntuales insesgados y eficientes para las variables macroeconómicas clave.")
print("b). Validez estadística: Los estimadores cumplen con las propiedades deseables de insesgadez, consistencia y eficiencia.")
print("c). Relevancia económica: Los estimadores proporcionan información valiosa sobre los parámetros poblacionales de interés.")
print("d). Modelo econométrico: Los estimadores del modelo sugieren relaciones significativas entre las variables, proporcionando evidencia para evaluar nuestra hipótesis de investigación.")
print("e). Implicaciones: Los resultados indican que tanto el crecimiento del PIB como la inflación tienen efectos medibles en la tasa de desempleo, lo cual es consistente con la teoría económica.")

## 2. Métodos de remuestreo: Bootstrap

**1**

El método Bootstrap nos permite, a base de realizar varios análisis con los datos disponibles, predecir lo que va a suceder al repetir el mismo experimento varias veces. 

Este fue inventado por Bradley Efron en 1979, cuyo nombre proviene de la frase to pull oneself up by one’s bootstrap, la cual representa la capacidad de poder salir adelante por los medios propios. Su motivación fue la de evitar que los resultados fueran alterados por fuertes supuestos, pudiendo así generar resultados más precisos con las muestras disponibles, provocando que sea muy versátil e importante en la estadística moderna. 

**2**

El método consiste en los siguientes pasos: 

- Contar el número de muestras analizadas y analizar alguna de las medidas estadísticas (media, promedio, moda, entre otras) 

- Crear una nueva colección de muestras con el mismo número de muestras que las originales, pudiendo seleccionar varias veces la misma muestra   

- Analizar la medida estadística realizada en la original y volver a hacerlo con la nueva colección

- Repetir estos dos últimos pasos muchas veces 

- Crear un histograma que permita analizar la varianza, sesgo e intervalo de confianza, simulando así que se haya repetido el mismo experimento varias veces 

**3**

In [ ]:
from scipy.stats import gaussian_kde

def bootstrap_economico(data, statistic_func, n_bootstrap=1000, alpha=0.05):
    n = len(data)
    bootstrap_stats = []
    
    # Estadístico original
    original_stat = statistic_func(data)
    
    # Generar muestras bootstrap
    for i in range(n_bootstrap):
        # Muestreo con reemplazo
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        # Calcular estadístico
        stat = statistic_func(bootstrap_sample)
        bootstrap_stats.append(stat)
    
    bootstrap_stats = np.array(bootstrap_stats)
    bias = np.mean(bootstrap_stats) - original_stat
    std_error = np.std(bootstrap_stats, ddof=1)
    
    # Intervalos de confianza percentil
    lower_percentile = np.percentile(bootstrap_stats, (alpha/2)*100)
    upper_percentile = np.percentile(bootstrap_stats, (1-alpha/2)*100)
    
    # Intervalo de confianza normal - AQUÍ SE USA norm
    z_critical = norm.ppf(1 - alpha/2)
    lower_normal = original_stat - z_critical * std_error
    upper_normal = original_stat + z_critical * std_error
    
    return {
        'original': original_stat,
        'bootstrap_mean': np.mean(bootstrap_stats),
        'bias': bias,
        'std_error': std_error,
        'ci_percentil': (lower_percentile, upper_percentile),
        'ci_normal': (lower_normal, upper_normal),
        'bootstrap_distribution': bootstrap_stats
    }

print("Aplicación a Variables Económicas Clave:")

print()

variables_bootstrap = {
    'GDP Growth (% Annual)': 'Crecimiento Económico',
    'Unemployment Rate (%)': 'Tasa de Desempleo', 
    'Inflation (CPI %)': 'Tasa de Inflación',
    'GDP per Capita (Current USD)': 'PIB per Cápita'
}

resultados_bootstrap = {}

for variable, descripcion in variables_bootstrap.items():
    print(f"{descripcion} ({variable})")
    
    # Limpiar datos
    data_clean = df[variable].dropna()
    
    if len(data_clean) > 0:
        # Aplicar bootstrap para la media
        resultado = bootstrap_economico(data_clean, np.mean, n_bootstrap=1000)
        resultados_bootstrap[variable] = resultado
        
        print(f"- Media original: {resultado['original']:.4f}")
        print(f"- Media bootstrap: {resultado['bootstrap_mean']:.4f}")
        print(f"- Sesgo estimado: {resultado['bias']:.4f}")
        print(f"- Error estándar bootstrap: {resultado['std_error']:.4f}")
        print(f"- IC 95% percentil: [{resultado['ci_percentil'][0]:.4f}, {resultado['ci_percentil'][1]:.4f}]")
        print(f"- IC 95% normal: [{resultado['ci_normal'][0]:.4f}, {resultado['ci_normal'][1]:.4f}]")
        print()
    else:
        print(f"- No hay datos suficientes para {variable}")
        print()

**4**

In [ ]:
print("Tabla Resumen - Estimaciones Bootstrap:")

print()

resumen_data = []
for variable, descripcion in variables_bootstrap.items():
    if variable in resultados_bootstrap:
        result = resultados_bootstrap[variable]
        resumen_data.append({
            'Indicador Económico': descripcion,
            'Media Original': f"{result['original']:.4f}",
            'Media Bootstrap': f"{result['bootstrap_mean']:.4f}",
            'Sesgo': f"{result['bias']:.6f}",
            'Error Estándar': f"{result['std_error']:.4f}",
            'IC Percentil 95%': f"[{result['ci_percentil'][0]:.3f}, {result['ci_percentil'][1]:.3f}]",
            'Amplitud IC': f"{(result['ci_percentil'][1] - result['ci_percentil'][0]):.3f}"
        })

resumen_df = pd.DataFrame(resumen_data)
print(resumen_df.to_string(index=False))
print()

print("Análisis de las Distribuciones Bootstrap:")
print()

for variable, descripcion in variables_bootstrap.items():
    if variable in resultados_bootstrap:
        result = resultados_bootstrap[variable]
        bootstrap_dist = result['bootstrap_distribution']
        
        print(f"{descripcion}:")
        print(f"- Rango bootstrap: [{np.min(bootstrap_dist):.4f}, {np.max(bootstrap_dist):.4f}]")
        # AQUÍ SE USA stats - CORREGIDO
        print(f"- Asimetría bootstrap: {stats.skew(bootstrap_dist):.4f}")
        print(f"- Curtosis bootstrap: {stats.kurtosis(bootstrap_dist):.4f}")
        print(f"- Coeficiente de variación: {(result['std_error'] / result['bootstrap_mean']) * 100:.2f}%")
        
        if 'Crecimiento' in descripcion:
            if result['ci_percentil'][0] > 0:
                print("- Significado económico: Crecimiento positivo estadísticamente significativo")
            else:
                print("- Significado económico: Crecimiento no significativamente diferente de cero")
        
        elif 'Desempleo' in descripcion:
            if result['ci_percentil'][1] < 5:
                print("- Significado económico: Tasa de desempleo baja (mercado laboral saludable)")
            elif result['ci_percentil'][0] > 10:
                print("- Significado económico: Tasa de desempleo alta (problemas estructurales)")
            else:
                print("- Significado económico: Tasa de desempleo moderada")
        
        elif 'Inflación' in descripcion:
            if result['ci_percentil'][0] > 2 and result['ci_percentil'][1] < 5:
                print("- Significado económico: Inflación en rango objetivo típico")
            elif result['ci_percentil'][0] > 10:
                print("- Significado económico: Inflación alta (presión inflacionaria)")
        
        print()

print("Visualización de Distribuciones Bootstrap")
print()

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for idx, (variable, descripcion) in enumerate(variables_bootstrap.items()):
    if variable in resultados_bootstrap:
        result = resultados_bootstrap[variable]
        bootstrap_dist = result['bootstrap_distribution']
        
        # Histograma de la distribución bootstrap
        n, bins, patches = axes[idx].hist(bootstrap_dist, bins=30, alpha=0.7, 
                                        color=colors[idx], edgecolor='black', 
                                        density=True)
        
        # Líneas de referencia - CORREGIDO
        original_label = f'Media original: {result["original"]:.3f}'
        bootstrap_label = f'Media bootstrap: {result["bootstrap_mean"]:.3f}'
        
        axes[idx].axvline(result['original'], color='red', linestyle='--', 
                         linewidth=3, label=original_label)
        axes[idx].axvline(result['bootstrap_mean'], color='blue', linestyle='--',
                         linewidth=2, label=bootstrap_label)
        
        # Intervalos de confianza
        axes[idx].axvline(result['ci_percentil'][0], color='green', linestyle=':', 
                         linewidth=2, alpha=0.8, label='Límites IC 95%')
        axes[idx].axvline(result['ci_percentil'][1], color='green', linestyle=':', 
                         linewidth=2, alpha=0.8)
        
        # Área de intervalo de confianza
        axes[idx].axvspan(result['ci_percentil'][0], result['ci_percentil'][1], 
                         alpha=0.2, color='green', label='IC 95%')
        
        # Curva de densidad suavizada - AQUÍ SE USA gaussian_kde (ya importado)
        kde = gaussian_kde(bootstrap_dist)
        x_vals = np.linspace(bins[0], bins[-1], 100)
        axes[idx].plot(x_vals, kde(x_vals), 'black', linewidth=1.5, label='Densidad')
        
        axes[idx].set_title(f'Distribución Bootstrap: {descripcion}', fontweight='bold', fontsize=12)
        axes[idx].set_xlabel('Valor del Estadístico', fontweight='bold')
        axes[idx].set_ylabel('Densidad de Probabilidad', fontweight='bold')
        axes[idx].legend(fontsize=9)
        axes[idx].grid(True, alpha=0.3)
        
        # Agregar estadísticas en el gráfico
        textstr = f'Sesgo: {result["bias"]:.4f}\nError std: {result["std_error"]:.4f}'
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
        axes[idx].text(0.02, 0.98, textstr, transform=axes[idx].transAxes, fontsize=9,
                      verticalalignment='top', bbox=props)

plt.suptitle('Distribuciones Bootstrap de Indicadores Macroeconómicos\n(1000 réplicas bootstrap)', 
             fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**5**

Analizando los datos, podemos concluir que son bastante similares entre sí, lo que confirma que ambos métodos son igual de válidos. Sin embargo, el bootstrapping es más confiable en este caso, ya que, este permite que la inferencia sea más creíble al analizar los distintos países y sus economías. Además, los resultados entregados por este método son más apegados a la realidad, simulando correctamente lo que pasaría si se repitiera la recolección de datos varias veces. El único caso donde la estimación clásica tiene ventaja es cuando los supuestos son totalmente acertados, permitiendo que el proceso sea más rápido en comparación con la estimación mediante bootstrap.

**¿Existen diferencias relevantes?**
No existen diferencias relevantes entre los resultados obtenidos en estimación puntual y Bootstrap , estas difieren en menos de  0.01 % en las medias y coincidencia en los intervalos de confianza.
Esto sugiere que los estimadores son insesgados, consistentes y eficientes, incluso bajo violaciones de normalidad como vimos en el estimador puntual.
La única diferencia fue en la Inflación donde el IC bootstrap es ligeramente más amplio debido a la asimetría positiva de su distribución, además de comfirmar el comportamiento volátil entre países.

**¿Qué ventajas aporta el bootstrap en el contexto de su análisis y qué limitaciones pueden identificarse?**
No depende de supuestos de normalidad, el bootstrap ofrece IC válidos sin asumir distribución normal, en nuestro caso al realizar test de Shapiro-Wilk rechazo normalidad.
Evalúa el sesgo empírico al poder cuatificar estimadores puntuales tienden a sobre- o sub-estimar el valor real.
Limitaciones:
Alta demanda computacional al tener que repetir la muestra muchas veces.
El bootstrap repite el sesgo si la muestra original no representa bien la población, generando dependencia.

Por lo que en nuestro caso, fue una herramienta valiosa al comprobar que los estimadores son insesgados, consistentes y eficientes.